# Multi-agent Customer Support Automation

Six elements which help make Agents perform even better:
- Role Playing
- Focus
- Tools
- Cooperation
- Guardrails
- Memory

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from crewai import Agent, Task, Crew, LLM

In [14]:
from utils import get_gemini_api_key

llm = LLM(
    model="gemini/gemini-1.5-pro-latest",
    api_key=get_gemini_api_key(),
    temperature=0.2,
)

## Role Playing, Focus and Cooperation

In [4]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True,
    llm=llm
)

In [5]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True,
    llm=llm
)

## Tools, Guardrails and Memory

### Tools

- Import CrewAI tools

In [6]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

/home/zaeem/Courses/Multi AI Agent Systems with crewAI/venv/lib/python3.10/site-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/home/zaeem/Courses/Multi AI Agent Systems with crewAI/venv/lib/python3.10/site-packages/crewai_tools/tools/file_writer_tool/file_writer_tool.py:2: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.util import strtobool
/home/zaeem/Courses/Multi AI Agent Systems with crewAI/venv/lib/python3.10/site-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. Y

### Possible Custom Tools
- Load customer data
- Tap into previous conversations
- Load data from a CRM
- Checking existing bug reports
- Checking existing feature requests
- Checking ongoing tickets
- ... and more

In [7]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

### Creating Tasks
- Passing the Tool on the Task Level.

In [8]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

In [9]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [15]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=True,
  # memory=True
)

### Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

#### Guardrails
- By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [16]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 44600341-8a19-4090-bd41-2b8df53aecfc                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: cc75d55d-e738-4bcc-884d-640b1b6e784c
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: cc75d55d-e738-4bcc-884d-640b1b6e784c
       Status: Executing Task...
    └── 🤖 Agent: Senior Support Representative
            Status: In Progress

# Agent: Senior Support Representative
## Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


🚀 Crew: crew
└── 📋 Task: cc75d55d-e738-4bcc-884d-640b1b6e784c
       Status: Executing Task...
    └── 🤖 Agent: Senior Support Representative
            Status: In Progress
        └── 🧠 Thinking...

...



# Agent: Support Quality Assurance Specialist
## Final Answer: 
Hi Andrew, thanks for reaching out!

We appreciate you reaching out about setting up memory for your Crews. While CrewAI doesn't have a standalone "memory" feature called out specifically, we offer several powerful ways to incorporate knowledge and context, effectively achieving the same outcome.  Let's break down how you can achieve this:

* **Knowledge LLMs:** You're spot on about Knowledge LLMs! These are Large Language Models pre-trained or fine-tuned on specific knowledge domains.  By selecting the right Knowledge LLM for your Crew, you're essentially giving it a built-in knowledge base.  This is perfect for situations where you need your Crew to have specialized knowledge from the get-go.  Think of it like choosing the right expert for the job.  You can find more details on selecting and managing LLMs within CrewAI here: [link to relevant documentation on LLMs].

* **Tools (This is KEY!):**  CrewAI's strength lies 

🚀 Crew: crew
├── 📋 Task: cc75d55d-e738-4bcc-884d-640b1b6e784c
│      Assigned to: Senior Support Representative
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Support Representative
│           Status: ✅ Completed
│       └── 🔧 Used Read website content (1)
└── 📋 Task: a60940a4-d1ec-4345-b483-3cc036436004
       Status: Executing Task...
    └── 🤖 Agent: Support Quality Assurance Specialist
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: cc75d55d-e738-4bcc-884d-640b1b6e784c
│      Assigned to: Senior Support Representative
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Support Representative
│           Status: ✅ Completed
│       └── 🔧 Used Read website content (1)
└── 📋 Task: a60940a4-d1ec-4345-b483-3cc036436004
       Assigned to: Support Quality Assurance Specialist
       Status: ✅ Completed
    └── 🤖 Agent: Support Quality Assurance Specialist
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: a60940a4-d1ec-4345-b483-3cc036436004                                                                     │
│  Agent: Support Quality Assurance Specialist                                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 44600341-8a19-4090-bd41-2b8df53aecfc                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [17]:
from IPython.display import Markdown
Markdown(result.raw)

Hi Andrew, thanks for reaching out!

We appreciate you reaching out about setting up memory for your Crews. While CrewAI doesn't have a standalone "memory" feature called out specifically, we offer several powerful ways to incorporate knowledge and context, effectively achieving the same outcome.  Let's break down how you can achieve this:

* **Knowledge LLMs:** You're spot on about Knowledge LLMs! These are Large Language Models pre-trained or fine-tuned on specific knowledge domains.  By selecting the right Knowledge LLM for your Crew, you're essentially giving it a built-in knowledge base.  This is perfect for situations where you need your Crew to have specialized knowledge from the get-go.  Think of it like choosing the right expert for the job.  You can find more details on selecting and managing LLMs within CrewAI here: [link to relevant documentation on LLMs].

* **Tools (This is KEY!):**  CrewAI's strength lies in its ability to connect your agents with a wide array of tools, including access to external data sources.  This is your primary route to providing persistent "memory" for your Crews. Here's how:

    * **RAG Search & CSV RAG Search:** These tools allow your agents to query and retrieve information from your own documents and CSV files.  Imagine having a massive knowledge base at your Crew's fingertips!  They can search through this information in real-time, pulling up relevant details as needed.  [link to relevant documentation on RAG Search]

    * **Database Connectors:**  For more structured data, we offer connectors to various databases.  This allows your Crew to directly access and utilize information stored in your databases, providing a dynamic and up-to-date knowledge source. [link to relevant documentation on Database Connectors]

    * **Other Tools:**  We're constantly expanding our toolset!  Explore the full list of available tools to see what else might be useful for your specific use case. [link to the main tools documentation page]

* **Flows and State Management:** Flows are crucial for orchestrating complex workflows.  The state management within Flows allows you to store and access information generated during the workflow's execution.  This acts as a short-term memory, allowing your agents to remember and utilize information from previous steps in the flow.  For example, if a user provides their name in one step, that information can be stored in the flow's state and used to personalize interactions later on. [link to relevant documentation on Flows and State Management]


To get started, I highly recommend checking out the Tools section of our documentation.  It'll walk you through how to integrate these powerful tools into your Crew and give you a clearer picture of how to best leverage them for your specific needs.

We're excited to see what you build with DeepLearningAI's Crew!  If you have any more questions or need more specific guidance, please don't hesitate to ask.  We're here to support you every step of the way.